In [1]:
import time
time.localtime().tm_year

2016

In [2]:
week_to_msecs=7*24*3600*1000
week_to_msecs

604800000

In [3]:
#starting time: 01/25/2016 (Monday)
start_time=1453698000000

In [4]:
import os
os.chdir("/home/composersyf/Documents/RelishMBA/MixPanel_Data")

In [5]:
import json

with open("All_Events_Mixpanel_08_28.json") as events_data:
    events_data=events_data.readlines()[0]
events=json.loads(events_data) #events is stored as a list

In [6]:
import numpy as np
import pandas as pd

In [7]:
# Fix the data problems (company having multiple names) within the Events Data
for k,e in enumerate(events):
    try:
        if e['properties']['company_name']=="PwC's Strategy&" or e['properties']['company_name']=="PwC Consulting":
            events[k]['properties']['company_name']="PwC"
        elif e['properties']['company_name']=="Nike":
            events[k]['properties']['company_name']="Nike Inc."
        elif e['properties']['company_name']=='JP Morgan Chase & Co.':
            events[k]['properties']['company_name']="JP Morgan"
    except KeyError:
        pass
    try:
        if e['properties']['name']=="Premier":
            events[k]['properties']['name']="Premier, Inc."
        elif e['properties']['name']=="PwC's Strategy&" or e['properties']['name']=="PwC Consulting":
            events[k]['properties']['name']="PwC"
        elif e['properties']['name']=="Nike":
            events[k]['properties']['name']="Nike Inc."
        elif e['properties']['name']=="JP Morgan Chase & Co.":
            events[k]['properties']['name']="JP Morgan"
    except KeyError:
        pass

In [8]:
company_ids_names=[]
for e in events:
    if e['name']=="view Profile":
        company_ids_names.append((e['properties']["company_id"],e['properties']['company_name']))
company_ids_names_dict={}
for pair in company_ids_names:
    try:
        if pair[1] not in company_ids_names_dict[pair[0]]:
            company_ids_names_dict[pair[0]].append(pair[1])
    except KeyError:
        company_ids_names_dict[pair[0]]=[pair[1]]

In [9]:
#inverse dictionary
company_names_ids_dict={}
for pair in company_ids_names:
    company_names_ids_dict[pair[1]]=pair[0]
company_names_ids_dict

{'3M': 14,
 'Accenture': 17,
 'Alcoa Inc.': 90,
 'American Express': 126,
 'ArcheMedX': 164,
 'Archer Roose': 160,
 'BB&T': 71,
 'Bain & Company': 13,
 'Barclays': 193,
 'BizCorps': 81,
 'Boston Consulting Group': 33,
 'Castle Hill Gaming': 163,
 'Chevron': 50,
 'Cigna': 74,
 'Citi': 191,
 'Consulting Industry': 24,
 'Consumer Products Industry': 28,
 'Corning Inc.': 16,
 'Crane': 68,
 'DaVita Healthcare Partners': 190,
 'DuPont': 15,
 'E&J Gallo': 42,
 'Emerson': 63,
 'Environmental Defense Fund (EDF)': 145,
 'Evercore': 46,
 'Finance Industry': 25,
 'Forcier Consulting': 84,
 'Ford Motor Company': 72,
 'Genentech': 54,
 'Greystar': 96,
 'Guesthouse Reservation Services (Cassimir Club)': 165,
 'Harris Williams': 20,
 'Humana': 60,
 'JP Morgan': 182,
 'Jet.com': 110,
 'KiwiTech': 88,
 "L'Oréal": 134,
 'LLJ Capital': 171,
 'Livingstone Partners': 98,
 'M&T Bank': 57,
 'Mastercard': 37,
 'McKinsey & Company': 43,
 'Medtronic': 188,
 'Microsoft': 192,
 'Morgan Stanley': 40,
 'Morningstar'

In [10]:
exclude_list=['Accenture','Barclays','Boston Consulting Group','Citi','DaVita Healthcare Partners','Microsoft']
for e in exclude_list:
    del company_names_ids_dict[e]
company_names_ids_dict

{'3M': 14,
 'Alcoa Inc.': 90,
 'American Express': 126,
 'ArcheMedX': 164,
 'Archer Roose': 160,
 'BB&T': 71,
 'Bain & Company': 13,
 'BizCorps': 81,
 'Castle Hill Gaming': 163,
 'Chevron': 50,
 'Cigna': 74,
 'Consulting Industry': 24,
 'Consumer Products Industry': 28,
 'Corning Inc.': 16,
 'Crane': 68,
 'DuPont': 15,
 'E&J Gallo': 42,
 'Emerson': 63,
 'Environmental Defense Fund (EDF)': 145,
 'Evercore': 46,
 'Finance Industry': 25,
 'Forcier Consulting': 84,
 'Ford Motor Company': 72,
 'Genentech': 54,
 'Greystar': 96,
 'Guesthouse Reservation Services (Cassimir Club)': 165,
 'Harris Williams': 20,
 'Humana': 60,
 'JP Morgan': 182,
 'Jet.com': 110,
 'KiwiTech': 88,
 "L'Oréal": 134,
 'LLJ Capital': 171,
 'Livingstone Partners': 98,
 'M&T Bank': 57,
 'Mastercard': 37,
 'McKinsey & Company': 43,
 'Medtronic': 188,
 'Morgan Stanley': 40,
 'Morningstar': 116,
 'National Microfinance Bank': 99,
 'Nestle': 148,
 'Nike Inc.': 100,
 'Palantir Technologies': 149,
 'Peer Insight': 78,
 'PepsiC

### (1) Company Page Number of Visits by Student Users

In [11]:
number_of_weeks=(time.time()*1000-start_time)//week_to_msecs
number_of_weeks

31.0

In [12]:
page_visit_and_time={}
for e in events:
    if e['name']=="view Profile":
        try:
            page_visit_and_time[e['properties']['company_name']][(e['time']-start_time)//week_to_msecs]+=1
        except KeyError:
            page_visit_and_time[e['properties']['company_name']]=[0]*int(number_of_weeks)

In [13]:
for e in exclude_list:
    del page_visit_and_time[e]

In [14]:
weekly_page_visits=pd.DataFrame.from_dict(page_visit_and_time).T
weekly_page_visits

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
3M,4,2,3,3,2,2,7,14,22,13,...,15,21,19,11,13,9,11,12,17,15
Alcoa Inc.,2,6,2,0,2,0,0,4,6,8,...,0,3,0,2,0,0,0,0,0,1
American Express,0,0,0,0,0,0,0,0,0,0,...,12,22,8,18,20,14,35,22,20,10
ArcheMedX,0,0,0,0,5,0,0,2,1,6,...,2,3,3,1,0,0,1,0,0,0
Archer Roose,0,0,0,2,4,2,0,2,2,4,...,0,2,1,1,1,2,0,0,0,0
BB&T,6,1,0,3,0,0,4,4,5,3,...,5,3,2,4,1,3,6,9,6,9
Bain & Company,4,8,8,4,8,2,6,2,19,12,...,18,23,10,13,7,16,17,21,16,13
BizCorps,4,10,0,1,3,1,2,0,5,8,...,2,3,6,2,1,4,2,2,7,3
Castle Hill Gaming,0,0,0,0,1,0,0,3,1,3,...,1,2,0,1,0,0,0,0,1,0
Chevron,2,4,1,2,1,1,1,3,13,13,...,3,6,10,5,3,3,10,3,9,5


### (2) Company Profile Number of Saves by Student Users

In [15]:
events_time_order={}
for e in events:
    if e['name']=="Company Save" or e['name']=="UnSave Company":
        try:
            events_time_order[(e['time']-start_time)//week_to_msecs].append((e['time'],e))
        except KeyError:
            events_time_order[(e['time']-start_time)//week_to_msecs]=[(e['time'],e)]
for k in events_time_order:
    events_time_order[k]=np.array(events_time_order[k])[np.argsort([events_time_order[k][i][0] for i in range(len(events_time_order[k]))])]

In [16]:
company_saves_list={}
company_saves_by_week={}
for k in sorted(events_time_order.keys()):
    for e in events_time_order[k]:
        try:
            company_dict=company_saves_list[e[1]['properties']['company_name']]
        except KeyError:
            company_saves_list[e[1]['properties']['company_name']]={}
            company_dict=company_saves_list[e[1]['properties']['company_name']]
        if e[1]['name']=="Company Save":
            value=1
        else:
            value=0
        try:
            company_dict[e[1]["distinct_id"]].append(value)
        except KeyError:
            company_dict[e[1]["distinct_id"]]=[value]
    company_saves_by_week[k]={}
    for c in company_saves_list:
        company_saves_by_week[k][c]=sum([v[-1] for v in company_saves_list[c].values()])

In [17]:
weekly_company_saves=pd.DataFrame.from_dict(company_saves_by_week).fillna(0)
for c in range(weekly_company_saves.shape[1]):
    weekly_company_saves.iloc[:,c]=weekly_company_saves.iloc[:,c].astype(int)
weekly_company_saves

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
3M,2,4,4,6,6,6,6,8,16,20,...,57,61,66,68,70,73,76,80,85,86
Alcoa Inc.,0,1,1,1,1,1,1,1,3,3,...,4,4,4,4,4,4,4,4,4,4
American Express,0,0,0,0,0,0,0,0,0,0,...,13,21,23,25,28,30,33,36,39,40
ArcheMedX,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
Archer Roose,0,0,0,0,0,0,0,0,0,0,...,1,1,1,2,2,2,2,2,2,2
BB&T,3,3,3,4,4,4,4,4,5,5,...,15,16,16,16,16,18,21,22,24,24
Bain & Company,3,5,8,10,11,11,13,14,24,30,...,110,123,125,136,142,149,160,170,183,190
BizCorps,1,2,2,2,2,3,3,3,5,5,...,12,14,15,15,16,17,20,20,21,21
Castle Hill Gaming,0,0,0,0,0,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
Chevron,2,2,3,4,4,4,4,4,9,10,...,33,37,40,44,45,47,50,54,56,58


In [18]:
for c in range(weekly_company_saves.shape[1]-1,0,-1):
    weekly_company_saves.iloc[:,c]=weekly_company_saves.iloc[:,c]-weekly_company_saves.iloc[:,c-1]
weekly_company_saves

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
3M,2,2,0,2,0,0,0,2,8,4,...,0,4,5,2,2,3,3,4,5,1
Alcoa Inc.,0,1,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
American Express,0,0,0,0,0,0,0,0,0,0,...,3,8,2,2,3,2,3,3,3,1
ArcheMedX,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Archer Roose,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
BB&T,3,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,2,3,1,2,0
Bain & Company,3,2,3,2,1,0,2,1,10,6,...,4,13,2,11,6,7,11,10,13,7
BizCorps,1,1,0,0,0,1,0,0,2,0,...,1,2,1,0,1,1,3,0,1,0
Castle Hill Gaming,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chevron,2,0,1,1,0,0,0,0,5,1,...,1,4,3,4,1,2,3,4,2,2


### (3) Company Number of follows by Student Users

In [19]:
events_time_order_2={}
for e in events:
    if e['name']=="Company Follow" or e['name']=="Company UnFollow":
        try:
            events_time_order_2[(e['time']-start_time)//week_to_msecs].append((e['time'],e))
        except KeyError:
            events_time_order_2[(e['time']-start_time)//week_to_msecs]=[(e['time'],e)]
for k in events_time_order_2:
    events_time_order_2[k]=np.array(events_time_order_2[k])[np.argsort([events_time_order_2[k][i][0] for i in range(len(events_time_order_2[k]))])]

In [20]:
company_follows_list={}
company_follows_by_week={}
for k in sorted(events_time_order_2.keys()):
    for e in events_time_order_2[k]:
        try:
            company_dict=company_follows_list[e[1]['properties']['company_name']]
        except KeyError:
            company_follows_list[e[1]['properties']['company_name']]={}
            company_dict=company_follows_list[e[1]['properties']['company_name']]
        if e[1]['name']=="Company Follow":
            value=1
        else:
            value=0
        try:
            company_dict[e[1]["distinct_id"]].append(value)
        except KeyError:
            company_dict[e[1]["distinct_id"]]=[value]
    company_follows_by_week[k]={}
    for c in company_follows_list:
        company_follows_by_week[k][c]=sum([v[-1] for v in company_follows_list[c].values()])

In [21]:
weekly_company_follows=pd.DataFrame.from_dict(company_follows_by_week).fillna(0)
for c in range(weekly_company_follows.shape[1]):
    weekly_company_follows.iloc[:,c]=weekly_company_follows.iloc[:,c].astype(int)
weekly_company_follows=weekly_company_follows.loc[np.setdiff1d(weekly_company_follows.index,np.array(exclude_list)),:]
weekly_company_follows

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
3M,1,3,3,4,4,4,5,7,12,18,...,68,71,75,82,87,92,97,102,107,110
Alcoa Inc.,0,0,0,0,0,0,0,0,2,3,...,5,5,5,5,5,5,5,5,5,5
American Express,0,0,0,0,0,0,0,0,0,0,...,24,32,34,39,45,49,56,60,66,71
ArcheMedX,0,0,0,0,1,1,1,1,1,2,...,4,4,5,5,5,5,5,5,5,5
Archer Roose,0,0,0,0,1,2,2,2,2,2,...,4,4,5,6,6,6,6,6,6,7
BB&T,2,3,4,5,5,5,6,6,7,7,...,21,22,22,22,22,26,30,31,32,34
Bain & Company,2,4,4,6,9,10,10,12,25,34,...,152,165,171,181,193,203,219,229,243,252
BizCorps,0,2,2,2,2,3,3,3,4,5,...,16,19,21,22,23,27,29,30,30,34
Castle Hill Gaming,0,0,0,0,0,1,1,1,1,1,...,1,1,2,3,3,3,3,3,3,3
Chevron,3,3,3,3,4,6,6,6,10,13,...,45,47,50,57,59,64,68,71,75,75


In [22]:
for c in range(weekly_company_follows.shape[1]-1,0,-1):
    weekly_company_follows.iloc[:,c]=weekly_company_follows.iloc[:,c]-weekly_company_follows.iloc[:,c-1]
weekly_company_follows

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
3M,1,2,0,1,0,0,1,2,5,6,...,7,3,4,7,5,5,5,5,5,3
Alcoa Inc.,0,0,0,0,0,0,0,0,2,1,...,1,0,0,0,0,0,0,0,0,0
American Express,0,0,0,0,0,0,0,0,0,0,...,7,8,2,5,6,4,7,4,6,5
ArcheMedX,0,0,0,0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
Archer Roose,0,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
BB&T,2,1,1,1,0,0,1,0,1,0,...,3,1,0,0,0,4,4,1,1,2
Bain & Company,2,2,0,2,3,1,0,2,13,9,...,9,13,6,10,12,10,16,10,14,9
BizCorps,0,2,0,0,0,1,0,0,1,1,...,2,3,2,1,1,4,2,1,0,4
Castle Hill Gaming,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
Chevron,3,0,0,0,1,2,0,0,4,3,...,4,2,3,7,2,5,4,3,4,0


In [23]:
weekly_periods=[]
for i in range(int(number_of_weeks)):
    current_time=time.localtime(start_time/1000+i*week_to_msecs/1000)
    next_time=time.localtime(start_time/1000+(i+6/7)*week_to_msecs/1000)
    time_window=str(current_time.tm_year)+"/"+str(current_time.tm_mon).zfill(2)+"/"+str(current_time.tm_mday).zfill(2)+" - "+\
    str(next_time.tm_year)+"/"+str(next_time.tm_mon).zfill(2)+"/"+str(next_time.tm_mday).zfill(2)
    weekly_periods.append(time_window)
weekly_periods    

['2016/01/25 - 2016/01/31',
 '2016/02/01 - 2016/02/07',
 '2016/02/08 - 2016/02/14',
 '2016/02/15 - 2016/02/21',
 '2016/02/22 - 2016/02/28',
 '2016/02/29 - 2016/03/06',
 '2016/03/07 - 2016/03/13',
 '2016/03/14 - 2016/03/20',
 '2016/03/21 - 2016/03/27',
 '2016/03/28 - 2016/04/03',
 '2016/04/04 - 2016/04/10',
 '2016/04/11 - 2016/04/17',
 '2016/04/18 - 2016/04/24',
 '2016/04/25 - 2016/05/01',
 '2016/05/02 - 2016/05/08',
 '2016/05/09 - 2016/05/15',
 '2016/05/16 - 2016/05/22',
 '2016/05/23 - 2016/05/29',
 '2016/05/30 - 2016/06/05',
 '2016/06/06 - 2016/06/12',
 '2016/06/13 - 2016/06/19',
 '2016/06/20 - 2016/06/26',
 '2016/06/27 - 2016/07/03',
 '2016/07/04 - 2016/07/10',
 '2016/07/11 - 2016/07/17',
 '2016/07/18 - 2016/07/24',
 '2016/07/25 - 2016/07/31',
 '2016/08/01 - 2016/08/07',
 '2016/08/08 - 2016/08/14',
 '2016/08/15 - 2016/08/21',
 '2016/08/22 - 2016/08/28']

In [24]:
weekly_page_visits.columns=weekly_periods
weekly_company_saves.columns=weekly_periods
weekly_company_follows.columns=weekly_periods

In [25]:
weekly_page_visits.to_csv("/home/composersyf/Documents/RelishMBA/Shiny_dygraphs/Weekly_Page_Visits.csv", index=True, index_label="company_name")
weekly_company_saves.to_csv("/home/composersyf/Documents/RelishMBA/Shiny_dygraphs/Weekly_Company_Saves.csv", index=True, index_label="company_name")
weekly_company_follows.to_csv("/home/composersyf/Documents/RelishMBA/Shiny_dygraphs/Weekly_Company_Follows.csv", index=True, index_label="company_name")